In [1]:
import csv
import pandas as pd
import numpy as np
import keras
from keras import layers
from keras.models import Model
from keras import metrics
from keras import backend as K
import tensorflow as tf
# import sklearn
import glob
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import KFold, cross_val_score, train_test_split

path='TrainingData'
files = glob.glob('*__x.csv')
# files = os.listdir(path)
# print(files)
li = []

for filename in files:
    df = pd.read_csv(filename, header=None)
    li.append(df)
# print(li)
data = pd.concat(li)


data.iloc[:,:] = Normalizer(norm='l2').fit_transform(data)

x_movingwindows=[]
i=0
while i <=len(data)-160:
    frames=[data[i:i+160][0],data[i:i+160][1],data[i:i+160][2],data[i:i+160][3],data[i:i+160][4],data[i:i+160][5]]
    result=np.array(pd.concat(frames))
    x_movingwindows.append(result)
    frames=[]
    result=[]
    i=i+80
x_movingwindows = np.array(x_movingwindows)

# print(x_movingwindows[0:5])

#take the mode of the y
files_y = glob.glob('*__y.csv')
# print(files_y)
liy = []

for filename_y in files_y:
    df_y = pd.read_csv(filename_y, header=None)
    liy.append(df_y)

data_y = pd.concat(liy)
j=0
y_data=[]
while j <=len(data_y)-40:
    b=data_y[j:j+40].value_counts(ascending= False)
    a=list(b.index)
    y_data.append(a[0])
    j=j+20
# print(y_data)
# print(len(y_data))


#split to train and validaion
kfold = KFold(3, True, 1)
# x_train, x_validation,y_train,y_validation=kfold.split(data)
x_train, x_validation,y_train,y_validation = train_test_split(x_movingwindows,y_data,test_size=0.2, random_state = 5)


In [2]:
x_train

array([[-0.16941634, -0.26715459,  0.17229177, ..., -0.03604109,
         0.02954058,  0.0438414 ],
       [ 0.35443113,  0.01201069, -0.15396522, ...,  0.0237442 ,
         0.02460033,  0.00960306],
       [ 0.35003581, -0.37061222, -0.43531273, ...,  0.05135395,
        -0.13586048, -0.18025257],
       ...,
       [ 0.15371629,  0.46213246,  0.47135492, ..., -0.14850699,
        -0.10860008, -0.08247865],
       [-0.22505759, -0.03828126,  0.06934077, ..., -0.11043542,
        -0.10041926, -0.05348878],
       [-0.6398898 , -0.36112649, -0.09641084, ...,  0.08356353,
         0.06203936,  0.04745596]])

In [3]:
x_validation

array([[ 6.98717252e-01, -9.26817046e-02, -2.04546199e-01, ...,
         4.47210625e-02,  1.98821070e-02,  1.15918594e-02],
       [-3.65771554e-01, -8.59040043e-02,  2.01248174e-01, ...,
         1.82430104e-02, -7.26446652e-03,  5.46932558e-04],
       [-2.24203785e-01, -2.53259286e-01,  1.11270555e-01, ...,
        -2.90145015e-02, -4.55208874e-02, -4.89430640e-02],
       ...,
       [-1.60204759e-01,  5.66452336e-02, -3.77806972e-02, ...,
         1.40879028e-01,  1.42921318e-01,  1.45359398e-01],
       [ 4.92732294e-02, -7.84041806e-02, -1.23675208e-01, ...,
         3.49925233e-02,  3.32205978e-02,  2.85291532e-02],
       [ 2.06384252e-01,  1.47787129e-01, -1.93444075e-01, ...,
        -5.45301889e-02, -3.05611626e-02, -3.13711281e-02]])

In [4]:
x_train=x_train.reshape(len(x_train),6,160,-1)
print(x_train.shape)

(13415, 6, 160, 1)


In [5]:
x_validation=x_validation.reshape(len(x_validation),6, 160,-1)
print(x_validation.shape)

(3354, 6, 160, 1)


In [6]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
latent_dim = 2

encoder_inputs = keras.Input(shape=(6, 160, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu",  padding="same")(x)
x = layers.Conv2D(64, 3, activation="relu",  strides=1,padding="same")(x)
shape_before_flattening = K.int_shape(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 6, 160, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 3, 80, 32)    320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 3, 80, 64)    18496       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 3, 80, 64)    36928       conv2d_1[0][0]                   
____________________________________________________________________________________________

In [7]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(15360, activation="relu")(latent_inputs)
x = layers.Reshape((3,80,64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=1, padding="same")(x)
x = layers.Conv2D(64, 3, activation="relu",  padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 15360)             46080     
_________________________________________________________________
reshape (Reshape)            (None, 3, 80, 64)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 3, 80, 64)         36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 80, 64)         36928     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 6, 160, 32)        18464     
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 6, 160, 1)         289 

In [8]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data, reconstruction)
            )
            # reconstruction_loss *= 6*160
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -5e-4
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

In [9]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(x_train, epochs=30, batch_size=128)

Epoch 1/30
105/105 [==============================] - 32s 306ms/step - loss: 0.1216 - reconstruction_loss: 0.1206 - kl_loss: 0.0010
Epoch 2/30
105/105 [==============================] - 33s 315ms/step - loss: -216.2541 - reconstruction_loss: -729.1006 - kl_loss: 512.8465
Epoch 3/30
105/105 [==============================] - 38s 358ms/step - loss: -18450.8949 - reconstruction_loss: -100752.8982 - kl_loss: 82302.0026
Epoch 4/30
 13/105 [==>...........................] - ETA: 33s - loss: -16337.7462 - reconstruction_loss: -16479.8246 - kl_loss: 142.0786

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
# encoder = Model(x_train)
x_valid_encoded = encoder.predict(x_validation, batch_size=batch_size)
# acc = accuracy_score(x_valid_encoded, y_validation)
# print('Accuracy: %.3f' % acc)
plt.figure(figsize=(10, 10))
plt.scatter(x_valid_encoded[:, 0], x_valid_encoded[:, 1], c=y_validation, cmap='brg')
plt.colorbar()
plt.show()

In [11]:
import matplotlib.pyplot as plt
def plot_label_clusters(encoder, decoder, data, labels):
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = encoder.predict(data)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=labels)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()




plot_label_clusters(encoder, decoder, x_train, y_train)

AttributeError: 'list' object has no attribute 'shape'